> Aprenentatge Computacional

**Nom**: Gabriel Rios Sanchez\
**NIU**: 1671177

# SUPERMARKET SALES

---

In [2]:
import pandas as pd

datos = pd.read_csv("dataset/data.csv")
datos.isna().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64

## 0. ENUNCIAT INICIAL

---

## 1. ANÀLISI DE DADES

---

## 2. PREPROCESSAT

---

## 3. SELECCIÓ DE MODELS

---

## 4. CONCLUSIÓ